# Imports

In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-02-06 11:19:08.697464: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Processing

In [2]:
filelist = []

import os
for dirname, _, filenames in os.walk("/Users/alertrack/Documents/Captcha-Bot/dataset/archive"):
    for filename in filenames:        
        filelist.append(os.path.join(dirname, filename))

In [ ]:
labels = []
images = []
for image_path in filelist:
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = image / 255  # Normalização
    
    resized_image = cv2.resize(image, (150, 40))
    
    reshaped_image = np.expand_dims(resized_image, axis=-1)
    
    images.append(reshaped_image)
    
    image_label = image_path.split("/")[-1].split(".")[0]
    labels.append(image_label)

images = np.array(images)
labels = np.array(labels)

In [ ]:
images = np.stack(images)
print(images.shape)

In [ ]:
plt.imshow(images[19])
plt.title(labels[19])
plt.axis('on')
plt.show()

print(images[19])

In [ ]:
#One-Hot Encoding List of 5 character strings
def one_hot_encode(characters, char_pool):
    char_to_index = {char: i for i, char in enumerate(char_pool)}
    num_classes = len(char_pool)
    encoding_size = len(characters[0])
    
    one_hot_encoded = np.zeros((len(characters), encoding_size, num_classes), dtype=int)

    for i, word in enumerate(characters):
        for j, char in enumerate(word):
            index = char_to_index[char]
            one_hot_encoded[i, j, index] = 1

    return one_hot_encoded

char_pool = '0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [ ]:
#Encoding Labels
encoded_labels = one_hot_encode(labels, char_pool)
encoded_labels = np.array(encoded_labels)
encoded_labels.shape, encoded_labels

In [ ]:
#Shuffling Array
indices = np.random.permutation(len(labels))

images = images[indices]
encoded_labels = encoded_labels[indices]

images.shape, encoded_labels.shape

In [ ]:
#Train-Validation Split
split_index = int(len(labels) * 0.9)

train_labels = encoded_labels[:split_index]
val_labels = encoded_labels[split_index:]

train_images = images[:split_index]
val_images = images[split_index:]

In [ ]:
batch_size = 128

train_datagen = ImageDataGenerator(rescale=1.)
train_generator = train_datagen.flow(np.array(train_images), np.array(train_labels), batch_size=batch_size)

val_datagen = ImageDataGenerator(rescale=1.)
val_generator = val_datagen.flow(np.array(val_images), np.array(val_labels), batch_size=batch_size)

Creating Model

In [ ]:
# Convolutional Layer
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(40, 150, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))

In [ ]:
model.summary()

In [ ]:
#Dense Layer
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5 * 62, activation='softmax'))
model.add(layers.Reshape((5, 62)))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training

In [ ]:
history = model.fit(train_generator, epochs=30, validation_data=val_generator)

In [ ]:
loss_values = history.history['loss']
val_loss_values = history.history['val_loss']

plt.plot(loss_values, label='Training Loss')
plt.plot(val_loss_values, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.show()